In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from time import time
from plotly.subplots import make_subplots
from datastory import DataStory

from google.cloud import secretmanager
from google.cloud.bigquery import Client, LoadJobConfig

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
tuning = 10000

con = pesys_utils.open_pen_connection()
df_kontrollpunkt = pandas_utils.pandas_from_sql('../sql/kontrollpunkt.sql', con=con, tuning=tuning, lowercase=True)
con.close()

In [ ]:
df_kontrollpunkt.head()

In [ ]:
df_2023 = df_kontrollpunkt[df_kontrollpunkt.dato.dt.year == 2023]

In [ ]:
px.bar(df_2023, "dato", "antall", color="sakstype")

In [ ]:
client = Client(project="pensjon-saksbehandli-prod-1f83")

table_id = 'pensjon-saksbehandli-prod-1f83.kontrollpunkt.kontrollpunkt_daglig'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

start = time()
job = client.load_table_from_dataframe(df_kontrollpunkt, table_id, job_config=job_config)
job.result()

end = time()

print(f'{len(df_kontrollpunkt)} rad(er) ble skrevet til bigquery etter {end-start} sekunder.')